In [ ]:
#Some ways to create a bag of words

from nltk.corpus import wordnet
!pip install spacy
!python -m spacy download en_core_web_sm
!pip install gensim
project_base="/home/username/haiku/"

In [ ]:
syns = wordnet.synsets("plant_organ") 

In [ ]:
hypo = wordnet.synset("plant_organ.n.01").hyponyms()

In [ ]:
import spacy
import numpy as np
nlp = spacy.load('en_core_web_sm')

def most_similar(word, topn=5):
    word = nlp.vocab[str(word)]
    queries = [
        w for w in word.vocab 
        if w.is_lower == word.is_lower and w.prob >= -15 and np.count_nonzero(w.vector)
    ]

    by_similarity = sorted(queries, key=lambda w: word.similarity(w), reverse=True)
    return [(w.lower_,w.similarity(word)) for w in by_similarity[:topn+1] if w.lower_ != word.lower_]

In [ ]:
most_similar("sky", topn=7)

In [ ]:
for ss in wordnet.synsets('happiness'):
    print(ss.lemma_names())

In [ ]:
def synonyms(word, pos_tag):
  return list(
    {
      lemma.replace("_"," ").replace("-"," ") for synset in wordnet.synsets(
        _clean_word(word),
        pos_tag,
      ) for lemma in synset.lemma_names()
    }
  )

from string import punctuation
def _clean_word(word):
  return word.lower().strip(punctuation)

In [ ]:
synonyms("sky","n")

In [1]:
from nltk.corpus import brown
import gensim
model = gensim.models.Word2Vec(brown.sents())

In [2]:
model.most_similar(positive=['chest'], topn = 10)

<ipython-input-2-d14e1c37ded7>:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  model.most_similar(positive=['chest'], topn = 10)


[('shoulders', 0.9764670729637146),
 ('shoulder', 0.9759306907653809),
 ('sky', 0.9739558100700378),
 ('wagon', 0.9723237752914429),
 ('seat', 0.9685256481170654),
 ('knee', 0.9651641845703125),
 ('cheek', 0.9644531607627869),
 ('nose', 0.9642648100852966),
 ('pocket', 0.9636731147766113),
 ('mirror', 0.9618158340454102)]

In [3]:
from nltk.data import find
word2vec_sample = str(find('models/word2vec_sample/pruned.word2vec.txt'))
model = gensim.models.KeyedVectors.load_word2vec_format(word2vec_sample, binary=False)

In [4]:
model.most_similar(positive=['snake'], topn = 40)

[('snakes', 0.7513571977615356),
 ('python', 0.660629153251648),
 ('cobra', 0.648840606212616),
 ('rattlesnake', 0.6464701890945435),
 ('serpent', 0.6409705877304077),
 ('alligator', 0.6349284052848816),
 ('spider', 0.6340566873550415),
 ('crocodile', 0.6303909420967102),
 ('monkey', 0.6056618690490723),
 ('critter', 0.5975096225738525),
 ('toad', 0.5919240713119507),
 ('lizards', 0.5737593173980713),
 ('turtle', 0.5653021335601807),
 ('rattlesnakes', 0.5651679635047913),
 ('rat', 0.562048614025116),
 ('creature', 0.5596329569816589),
 ('possum', 0.5569279193878174),
 ('constrictor', 0.5555417537689209),
 ('serpents', 0.5516219735145569),
 ('fox', 0.5493873953819275),
 ('cottonmouth', 0.5457687377929688),
 ('rattlers', 0.5444181561470032),
 ('frog', 0.5431819558143616),
 ('rabbit', 0.5367448329925537),
 ('rodent', 0.5361548662185669),
 ('anaconda', 0.5355017185211182),
 ('rattler', 0.5352160930633545),
 ('cat', 0.5259495377540588),
 ('tortoise', 0.5164839029312134),
 ('squirrel', 0.516

In [5]:
res = model.most_similar(positive=['volcano'], topn = 100)

In [6]:
words = []
for re in res:
    words.append(re[0])

In [7]:
for word in words:
    print(word)

eruption
lava
volcanic
volcanos
crater
glacier
ash
plume
mountain
earthquakes
geysers
tremor
Krakatoa
glaciers
cave
ridge
landslides
erupting
tsunami
slopes
quake
earthquake
typhoon
Iceland
comet
mountainside
meteor
massifs
archipelago
Himalayas
undersea
waterfall
sunspot
Kamchatka
mountains
asteroid
Honshu
canyon
Sumatra
Pompeii
craters
ocean
equatorial
seaquake
mountainsides
Empedocles
Tikopia
caves
avalanche
meteorite
nebula
dome
epicenter
geologists
Celebes
meteors
sea
tropics
Antarctica
altitude
tropical
Tsunami
brushfire
slope
evacuation
telescope
mud
waterfalls
floe
ridges
seismographs
foothills
mountainous
comets
islands
seismological
cavern
cliff
hillside
storm
island
radioactivity
planetoid
boulders
boulder
quetzal
foothill
spewing
seismic
Moluccas
geological
kiloton
chimney
gusher
gullies
cliffs
erupt
equator
storms
hurricane


In [ ]:
#PERPLEXITY ANALYSIS FOLLOWS

In [1]:
import re
import math

# used for unseen words in training vocabularies
UNK = None
# sentence start and end
SENTENCE_START = "<s>"
SENTENCE_END = "</s>"

def read_sentences_from_file(file_path):
    with open(file_path, "r") as f:
        return [re.split("\s+", line.rstrip('\n')) for line in f]

class UnigramLanguageModel:
    def __init__(self, sentences, smoothing=False):
        self.unigram_frequencies = dict()
        self.corpus_length = 0
        for sentence in sentences:
            for word in sentence:
                self.unigram_frequencies[word] = self.unigram_frequencies.get(word, 0) + 1
                if word != SENTENCE_START and word != SENTENCE_END:
                    self.corpus_length += 1
        # subtract 2 because unigram_frequencies dictionary contains values for SENTENCE_START and SENTENCE_END
        self.unique_words = len(self.unigram_frequencies) - 2
        self.smoothing = smoothing

    def calculate_unigram_probability(self, word):
            word_probability_numerator = self.unigram_frequencies.get(word, 0)
            word_probability_denominator = self.corpus_length
            if self.smoothing:
                word_probability_numerator += 1
                # add one more to total number of seen unique words for UNK - unseen events
                word_probability_denominator += self.unique_words + 1
            return float(word_probability_numerator) / float(word_probability_denominator)

    def calculate_sentence_probability(self, sentence, normalize_probability=True):
        sentence_probability_log_sum = 0
        for word in sentence:
            if word != SENTENCE_START and word != SENTENCE_END:
                word_probability = self.calculate_unigram_probability(word)
                sentence_probability_log_sum += math.log(word_probability, 2)
        return math.pow(2, sentence_probability_log_sum) if normalize_probability else sentence_probability_log_sum                

    def sorted_vocabulary(self):
        full_vocab = list(self.unigram_frequencies.keys())
        full_vocab.remove(SENTENCE_START)
        full_vocab.remove(SENTENCE_END)
        full_vocab.sort()
        full_vocab.append(UNK)
        full_vocab.append(SENTENCE_START)
        full_vocab.append(SENTENCE_END)
        return full_vocab

class BigramLanguageModel(UnigramLanguageModel):
    def __init__(self, sentences, smoothing=False):
        UnigramLanguageModel.__init__(self, sentences, smoothing)
        self.bigram_frequencies = dict()
        self.unique_bigrams = set()
        for sentence in sentences:
            previous_word = None
            for word in sentence:
                if previous_word != None:
                    self.bigram_frequencies[(previous_word, word)] = self.bigram_frequencies.get((previous_word, word),
                                                                                                 0) + 1
                    if previous_word != SENTENCE_START and word != SENTENCE_END:
                        self.unique_bigrams.add((previous_word, word))
                previous_word = word
        self.unique__bigram_words = len(self.unigram_frequencies)

    def calculate_bigram_probabilty(self, previous_word, word):
        bigram_word_probability_numerator = self.bigram_frequencies.get((previous_word, word), 0)
        bigram_word_probability_denominator = self.unigram_frequencies.get(previous_word, 0)
        if self.smoothing:
            bigram_word_probability_numerator += 1
            bigram_word_probability_denominator += self.unique__bigram_words
        return 0.0 if bigram_word_probability_numerator == 0 or bigram_word_probability_denominator == 0 else float(
            bigram_word_probability_numerator) / float(bigram_word_probability_denominator)

    def calculate_bigram_sentence_probability(self, sentence, normalize_probability=True):
        bigram_sentence_probability_log_sum = 0
        previous_word = None
        for word in sentence:
            if previous_word != None:
                bigram_word_probability = self.calculate_bigram_probabilty(previous_word, word)
                bigram_sentence_probability_log_sum += math.log(bigram_word_probability, 2)
            previous_word = word
        return math.pow(2,
                        bigram_sentence_probability_log_sum) if normalize_probability else bigram_sentence_probability_log_sum

# calculate number of unigrams & bigrams
def calculate_number_of_unigrams(sentences):
    unigram_count = 0
    for sentence in sentences:
        # remove two for <s> and </s>
        unigram_count += len(sentence) - 2
    return unigram_count

def calculate_number_of_bigrams(sentences):
        bigram_count = 0
        for sentence in sentences:
            # remove one for number of bigrams in sentence
            bigram_count += len(sentence) - 1
        return bigram_count

# print unigram and bigram probs
def print_unigram_probs(sorted_vocab_keys, model):
    for vocab_key in sorted_vocab_keys:
        if vocab_key != SENTENCE_START and vocab_key != SENTENCE_END:
            print("{}: {}".format(vocab_key if vocab_key != UNK else "UNK",
                                       model.calculate_unigram_probability(vocab_key)), end=" ")
    print("")

def print_bigram_probs(sorted_vocab_keys, model):
    print("\t\t", end="")
    for vocab_key in sorted_vocab_keys:
        if vocab_key != SENTENCE_START:
            print(vocab_key if vocab_key != UNK else "UNK", end="\t\t")
    print("")
    for vocab_key in sorted_vocab_keys:
        if vocab_key != SENTENCE_END:
            print(vocab_key if vocab_key != UNK else "UNK", end="\t\t")
            for vocab_key_second in sorted_vocab_keys:
                if vocab_key_second != SENTENCE_START:
                    print("{0:.5f}".format(model.calculate_bigram_probabilty(vocab_key, vocab_key_second)), end="\t\t")
            print("")
    print("")

# calculate perplexty
def calculate_unigram_perplexity(model, sentences):
    unigram_count = calculate_number_of_unigrams(sentences)
    sentence_probability_log_sum = 0
    for sentence in sentences:
        try:
            sentence_probability_log_sum -= math.log(model.calculate_sentence_probability(sentence), 2)
        except:
            sentence_probability_log_sum -= float('-inf')
    return math.pow(2, sentence_probability_log_sum / unigram_count)

def calculate_bigram_perplexity(model, sentences):
    number_of_bigrams = calculate_number_of_bigrams(sentences)
    bigram_sentence_probability_log_sum = 0
    for sentence in sentences:
        try:
            bigram_sentence_probability_log_sum -= math.log(model.calculate_bigram_sentence_probability(sentence), 2)
        except:
            bigram_sentence_probability_log_sum -= float('-inf')
    return math.pow(2, bigram_sentence_probability_log_sum / number_of_bigrams)


In [31]:
dataset = read_sentences_from_file(project_base+"/tests/original_dataset.txt")
dataset_test = re.split("\s+", "<s> pigeons in the hollow + i return fire + to birdsong </s>".rstrip('\n')) #read_sentences_from_file(project_base+"/tests/one.txt")

#dataset_model_unsmoothed = BigramLanguageModel(dataset)
dataset_model_smoothed = BigramLanguageModel(dataset, smoothing=True)

#sorted_vocab_keys = dataset_model_unsmoothed.sorted_vocabulary()

print("PERPLEXITY == ")

print("unigram: ", calculate_unigram_perplexity(dataset_model_smoothed, dataset))
print("bigram: ", calculate_bigram_perplexity(dataset_model_smoothed, dataset))

print("")

print("PERPLEXITY  gendata== ")
print("unigram: ", calculate_unigram_perplexity(dataset_model_smoothed, dataset_test))
print("bigram: ", calculate_bigram_perplexity(dataset_model_smoothed, dataset_test))


PERPLEXITY == 
unigram:  545.1452097219028
bigram:  1433.5895234684365

PERPLEXITY  gendata== 
unigram:  4848426724.64201
bigram:  14007.189093549354


bigram:  14007.189093549354


In [18]:
#AMT DATA PROCESSING FOLLOWS

In [ ]:
import numpy
import csv
res = []
with open(project_base+'/bare_3.csv', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',', quotechar='"')
    for row in reader:
        np = numpy.asarray(row).astype(numpy.float)
        res.append(numpy.reshape(np, (20, 3)))
        #resh = numpy.reshape(np, (20, 3))
        
mean = numpy.mean(res, axis=0)

In [ ]:
numpy.savetxt(project_base+"/out3.csv", mean, delimiter=",")

In [ ]:
#QUANTITATIVE ANALYSIS OF DATASETS FOLLOWS

In [ ]:
import csv
import re
import base64

datasets = ['guttenberg', 'sballas', 'tempslibre']

for dataset in datasets:
    with open(project_base+'/dataset/'+dataset+'.csv', 'rb') as file:
        vocabulary = []
        i = 0
        for line in file:
            if isinstance(line, (bytes, bytearray)):
                line = line.decode()
            line = re.sub(" , [0-2][0-9]:[0-5][0-9]", "", str(line))
            line = re.sub("[,|!|.|?|\"]", " ", line)
            line = line.replace("_","")
            line = line.replace("'"," ")
            line = line.replace(";","")
            line = line.replace("-","")
            line = line.replace("—","")
            line = line.replace("~","")
            line = line.replace("(","")
            line = line.replace(")","")
            words = [w.lower() for w in line.split()]
            for word in words:
                if word in vocabulary:
                    break
                else:
                    vocabulary.append(word)
                    i = i +1
    print (dataset+": " +str(i))
    #print(vocabulary)

In [ ]:
import csv
import re
import base64
import nltk

datasets = ['guttenberg', 'sballas', 'tempslibre']
#datasets = ['sballas']

for dataset in datasets:
    with open(project_base+'/dataset/'+dataset+'.csv', 'rb') as file:
        corpus = ""
        for line in file:
            if isinstance(line, (bytes, bytearray)):
                line = line.decode()
            line = re.sub(" , [0-2][0-9]:[0-5][0-9]", "", str(line))
            line = re.sub("[,|!|.|?|\"]", " ", line)
            line = line.replace("_","")
            line = line.replace("'"," ")
            line = line.replace(";","")
            line = line.replace("-","")
            line = line.replace("—","")
            line = line.replace("~","")
            line = line.replace("(","")
            line = line.replace(")","")        
            corpus = corpus + " " + line
            #print()
        text = nltk.word_tokenize(corpus)
        postag = nltk.pos_tag(text)
        tag_fd = nltk.FreqDist(tag for (word, tag) in postag)
        print (dataset+": ")
        #print(tag_fd.most_common())
        for word, frequency in tag_fd.most_common(50):
            print(u'{}, {}'.format(word, frequency))